In [84]:
from pdf2image import convert_from_path

In [85]:
pages = convert_from_path(r'docs\patient_details\pd_1.pdf',poppler_path=r'C:\poppler-24.02.0\Library\bin')

In [86]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398>]

In [87]:
pages[0].show()

In [88]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0], lang='eng')
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



In [89]:
import cv2
import numpy as np
def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR) # noqa
    processed_img = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 61, 11) # noqa
    return processed_img

In [90]:
img = preprocess_image(pages[0])

from PIL import Image
Image.fromarray(img).show()


text = pytesseract.image_to_string(img, lang='eng')
print(text)

17/12/2020

Patient Medical Record . : :

Patient Information

Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight:
9264 Ash Dr 95
New York City, 10005 a
United States Height:
190
In Case of Emergency
en oe
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
I a
Chicken Pox (Varicella): Measies:
IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



<h2>Extract Name</h2>

In [102]:
import re

pattern = r'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches


['\n\nBirth Date\nKathy Crawford May 6 1972\n']

In [92]:
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [93]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [94]:
import re

pattern = r'((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'

match = "Your text with dates like Jan 1, Feb 23, March 15"
date_matches = re.findall(pattern, match)
date = date_matches[0][0]

print(date)


Jan 1


In [95]:
match.replace(date, '').strip()

'Your text with dates like , Feb 23, March 15'

In [96]:
import re

def remove_noise_from_name(name):
    # Remove the 'Birth Date' text
    name = name.replace('Birth Date', '').strip()
    
    # Define the date pattern
    date_pattern = r'((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    
    # Find all date matches in the name
    date_matches = re.findall(date_pattern, name)
    
    if date_matches:
        # Get the first matched date
        date = date_matches[0][0]
        # Remove the date from the name
        name = name.replace(date, '').strip()
    
    return name


In [97]:
name = '\n\n \n\n \n\nBirth Date\nKathy Crawford May 6 1972\n'

name = remove_noise_from_name(name)
name

'Kathy Crawford'

<h2>Extract Phone</h2>

In [98]:
pattern = 'Patient Information(.*?)(\\(\\d{3}\\) \\d{3}-\\d{4})'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[('\n\nBirth Date\nKathy Crawford May 6 1972\n', '(737) 988-0851')]

In [99]:
matches[0][1]

'(737) 988-0851'

<h2>Extract Vaccine</h2>

In [100]:
pattern = r'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text, flags=re.DOTALL)
print(matches)

['No']


<h2>Extract Medical Problems</h2>

In [101]:
import re
pattern = r'List any Medical Problems \(asthma, seizures, headaches\):(.*)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Migraine'